In [ ]:
# numpy and pandas will be used for data manipulation
import numpy as np
import pandas as pd

# matplotlib will be used for visually representing our data
import matplotlib.pyplot as plt

# Yfinance will be used for importing historical oil prices
import yfinance as yfin

In [ ]:
# Setting our ticker
ticker = 'AMZN'
ticker = yfin.Ticker(ticker)

# Importing our data
data = ticker.history(period='5y')

In [ ]:
# Setting the text on the Y-axis
plt.ylabel("Amazon Stock Prices")

# Setting the size of our graph
data['Close'].plot(figsize=(10,5))

In [ ]:
data['MA_20'] = data['Close'].shift(1).rolling(window=20).mean()
data['MA_50']= data['Close'].shift(1).rolling(window=50).mean()
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()

In [ ]:
data.head(20)

In [ ]:
# Dropping the NaN values
data = data.dropna()

# Initialising X and assigning the two feature variables
X = data[['EMA_20','MA_50']]

# Getting the head of the data
X.head()

In [ ]:
# Dropping the NaN values
data = data.dropna()

# Initialising X and assigning the feature variables
X = data[['MA_20','EMA_20','MA_50', 'Close']]

# Getting the head of the data
X.head()

In [ ]:
# Setting the training set to 70% of the data
training = 0.7
t = int(training*len(X))

# Training dataset
data_train= X[:t]

# Testing dataset
data_test = X[t:]

In [ ]:
X[:t]

In [ ]:
X[t:]

In [ ]:
from pycaret.regression import *
s = setup(
    data=data[['MA_20','MA_50','EMA_20','Close']],
    target='Close',
    session_id=123,
    numeric_features=['MA_20','MA_50','EMA_20'],
    fold_strategy='timeseries', # Ensures temporal integrity
    data_split_shuffle=False,     # Prevents random shuffling
    n_jobs=-1
)

In [ ]:
# Create a Linear Regression model
lr = create_model('lr')

In [ ]:
tuned_lr = tune_model(lr)

In [ ]:
models()

In [ ]:
#Define Random Forest Regressor
rf = create_model('rf')

In [ ]:
#Train & Test Random Forest Regressor
tuned_rf = tune_model(rf)


In [ ]:
print(tuned_rf)

In [ ]:
plot_model(tuned_rf)

In [ ]:
plot_model(tuned_rf, plot = 'error')

In [ ]:
plot_model(tuned_rf, plot = 'feature')

In [ ]:
evaluate_model(tuned_rf)

In [ ]:
predict_model(tuned_rf)

In [ ]:
final_rf = finalize_model(tuned_rf)

In [ ]:
print(final_rf)

In [ ]:
predict_model(final_rf)

In [ ]:
unseenrf_predictions = predict_model(final_rf, data=data_test)
unseenrf_predictions

In [ ]:
#Visualise Random Forest Regressor
predictedrf_price = pd.DataFrame(unseenrf_predictions,index=unseenrf_predictions.index,columns = ['prediction_label'])
predictedrf_price.plot(figsize=(10,5))
unseenrf_predictions['Close'].plot()
plt.legend(['Predicted Price','Actual Price'])
plt.ylabel("Amazon Stock Prices")
plt.show()

#Comparison of performance against Linear Regression Model - See Cwk Documentation